In [ ]:

from prompt import system_prompt


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [11]:
import json
from openai import OpenAI
from tqdm import tqdm
import os
from dotenv import load_dotenv
import csv
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))    # <-------------본인 API키

folder_path = "C:/Users/USER/OneDrive/바탕 화면/내 자료/보아즈/ADV 플젝/소아협회50(20개만)_md"  # <------------전처리한 데이터 폴더

output_csv_path = "C:/Users/USER/OneDrive/바탕 화면/내 자료/보아즈/ADV 플젝/소아협회50(20개만)_qaset/PediatricAssociation.csv"                           # <---------------최종 저장 경로
csv_rows = []


In [12]:
system_prompt = """
# Role
You are a board-certified pediatric anesthesiologist tasked with generating medically accurate and clinically meaningful Korean Q&A data from pediatric anesthesia documents.

# Objective
Generate a high-quality QA dataset to fine-tune a Korean-language LLM specialized in pediatric anesthesia. The model will be used by clinical professionals, so questions must reflect what experienced physicians would realistically ask in real-world pediatric anesthesia settings.

# Core Principles
- Generate exactly **10** high-quality Q&A pairs
- Maintain strict medical accuracy and clinical validity
- Focus on practical and realistic clinical scenarios
- Ensure questions are specific, actionable, and non-redundant
- Include relevant numerical values when applicable
- Use professional and natural Korean suitable for clinical communication
- Generate questions **strictly based on the provided context**

# Question Generation Guidelines
1. **Clinical Relevance**
   - Focus on: diagnosis, treatment decisions, anesthesia drug usage, risk management, intra/post-op complications, airway management, fluid/blood dosing, sedation recovery
   - Avoid: basic memorization, definitions, or simple score lookups
   - Avoid questions derived from **tables** (e.g., “Ramsey sedation score 3점은 무엇인가요?”)
   - Avoid referencing the specific case (e.g., “본 연구에서”, “본 증례에서”) — all questions must be **generalizable**

2. **Non-Redundancy**
   - Do not include similar or overlapping questions
   - If multiple potential questions exist about the same topic, select only the **most clinically useful one**

3. **Question Format**
   - Open-ended, short-answer format
   - Written in natural, professional Korean
   - No multiple-choice or yes/no questions
   - Avoid awkward or repetitive phrasing

4. **Reference Requirements**
   - Include relevant support sentences as `"reference_sentences"` array
   - For figures: include captions and surrounding explanations
   - **Do not use content that appears only in tables**

5. **Answer Format**
   - Begin with a reasoning step using evidence from the context
   - Wrap quoted evidence in **##begin_quote## ... ##end_quote##**
   - End with a Korean full-sentence answer prefixed by `<ANSWER>:`

# Output Format
Return your output in this **strict JSON format**:
[
  {
    "question": "한국어로 된 의학 질문",
    "reference_sentences": [
      "관련 문장 1",
      "관련 문장 2"
    ],
    "answer": "##Reason: ...\n<ANSWER>: ..."
  }
]

# Quality Checks
- Verify all medical terminology and numerical values
- Check Korean grammar and phrasing
- Ensure JSON format validity
- Eliminate redundant or low-value questions

# Error Prevention
- Double-check medical terminology
- Verify numerical calculations
- Ensure proper Korean grammar
- Validate JSON structure
- Confirm context alignment

# Example Outputs
Return as a **valid JSON array** of objects. Each object must look like:

[
  {{
    "question": "5살 23kg 소아 환자의 마취 중 적절한 수액 주입량은?",
    "reference_sentences": [
      "20kg을 초과하는 소아의 유지 수액량은 첫 20kg에 대해 1500mL를 적용하고, 이후 1kg당 20mL를 추가로 계산한다.",
      "이 수액량은 전신마취 중 적절한 수분 공급을 위해 사용된다."
    ],
    "answer": "##Reason: 문서의 ##begin_quote## 20kg을 초과하는 소아의 유지 수액량은 첫 20kg에 대해 1500mL를 적용하고, 이후 1kg당 20mL를 추가로 계산한다 ##end_quote## 라는 설명에 따르면, 23kg 소아는 1500mL + (3×20mL) = 1560mL가 필요합니다.\n<ANSWER>: 5살 23kg 소아의 마취 중 유지 수액량은 1560mL입니다."
  }},
  {{
    "question": "소아 환자를 깨울 때 laryngospasm이 의심되면 어떤 처치를 해야 하나요?",
    "reference_sentences": [
      "Laryngospasm이 의심되는 경우 즉각적인 처치로는 jaw thrust, 양압 환기, 그리고 succinylcholine 투여가 포함된다.",
      "신속한 인식과 처치가 저산소증을 예방하는 데 중요하다."
    ],
    "answer": "##Reason: 문서에 따르면 ##begin_quote## 즉각적인 처치로는 jaw thrust, 양압 환기, 그리고 succinylcholine 투여가 포함된다 ##end_quote## 라고 되어 있습니다. 이는 환자의 기도를 유지하고 저산소증을 방지하는 데 중요한 조치입니다.\n<ANSWER>: Laryngospasm이 의심되는 경우 jaw thrust, 양압 환기, succinylcholine 투여를 포함한 즉각적인 처치가 필요합니다."
  }}
]
"""

In [13]:
for file_name in tqdm(os.listdir(folder_path)):

    file_path = os.path.join(folder_path, file_name)

    with open(file_path, 'r', encoding='utf-8') as f:
        data = f.read()

    try:
        response = client.chat.completions.create(
            model="gpt-4.1",  # 또는 gpt-4.0, gpt-3.5-turbo
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": data}
            ],
            temperature=0.2
        )
        
        output = response.choices[0].message.content.strip()
        qna_list = json.loads(output,strict=False)

        for qna in qna_list:
            question = qna['question']
            reference_sentences = qna['reference_sentences']
            answer = qna['answer']
            csv_rows.append([file_name, question, reference_sentences, answer])

    except Exception as e:
        print(f"❌ Error processing {file_name}: {e}")
        continue

# 5. 최종 CSV 저장
with open(output_csv_path, "w", newline='', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["파일명", "질문", "관련 문서", "답변"])  # 헤더
    writer.writerows(csv_rows)

print(f"✅ CSV 저장 완료: {output_csv_path}")

100%|██████████| 20/20 [08:36<00:00, 25.81s/it]

✅ CSV 저장 완료: C:/Users/USER/OneDrive/바탕 화면/내 자료/보아즈/ADV 플젝/소아협회50(20개만)_qaset/PediatricAssociation.csv


In [19]:
import pandas as pd

# CSV 파일 경로
file_path1 = "C:/Users/USER/OneDrive/바탕 화면/내 자료/보아즈/ADV 플젝/소아협회50(20개만)_qaset/PediatricAssociation.csv"

# CSV 파일 읽기
df1 = pd.read_csv(file_path1, encoding="utf-8")  # 또는 "cp949", "utf-8-sig" 등 필요에 따라 조정

# 저장할 Excel 파일 경로
excel_path = file_path1.replace(".csv", ".xlsx")

# Excel 파일로 저장
df1.to_excel(excel_path, index=False)
print(f"엑셀 파일로 저장 완료: {excel_path}")

엑셀 파일로 저장 완료: C:/Users/USER/OneDrive/바탕 화면/내 자료/보아즈/ADV 플젝/소아협회50(20개만)_qaset/PediatricAssociation.xlsx
